In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

In [ ]:
data = pd.read_csv('ijcai2016_taobao.csv')
data = data[data['act_ID'] == 1]
data = data.iloc[data['time'].argsort()]

In [ ]:
for iter in range(1):
    user_frequency = np.argsort(np.bincount(data.iloc[:, 0]))
    user_remained_index = np.zeros([len(user_frequency)])
    user_remained_index[user_frequency[-10000:]] = 1
    user_remained_index = {iter:user_remained_index[iter] for iter in range(len(user_frequency))}
    
    item_frequency = np.argsort(np.bincount(data.iloc[:, 2]))
    item_remained_index = np.zeros([len(item_frequency), ])
    item_remained_index[item_frequency[-10000:]] = 1
    item_remained_index = {iter:item_remained_index[iter] for iter in range(len(item_frequency))}
    
    data = data[data.iloc[:, 0].map(user_remained_index).astype(bool)]
    data = data[data.iloc[:, 2].map(item_remained_index).astype(bool)]

    data.iloc[:, 0], user = data.iloc[:, 0].factorize(sort=True)
    data.iloc[:, 2], item = data.iloc[:, 2].factorize(sort=True)

print('Number Users : {}'.format(len(user)))
print('Number Items : {}'.format(len(item)))

print('Density : {}'.format(data.shape[0] / (len(user) * len(item))))

data.iloc[:, 0] = data.iloc[:, 0].factorize(sort=True)[0]
data.iloc[:, 1] = data.iloc[:, 1].factorize(sort=True)[0]
data.iloc[:, 2] = data.iloc[:, 2].factorize(sort=True)[0]
data.iloc[:, 3] = data.iloc[:, 3].factorize(sort=True)[0]
data.iloc[:, 5] = data.iloc[:, 5].factorize(sort=True)[0]

In [ ]:
user_dict = {use_ID:[] for use_ID in data['use_ID']}

for row_id in tqdm(range(data.shape[0])):
    user = data.iloc[row_id, 0]
    seller = data.iloc[row_id, 1]
    item = data.iloc[row_id, 2]
    cate = data.iloc[row_id, 3]
    t = data.iloc[row_id, -1]
    user_dict[user].append([t, item, seller, cate])
    
for key in tqdm(user_dict):
    user_dict[key] = np.array(user_dict[key])
    user_dict[key] = user_dict[key][user_dict[key][:, 0].argsort()]
    new_time = pd.factorize(user_dict[key][:, 0], sort=True)[0][:, np.newaxis]
    user_dict[key] = np.concatenate([new_time, user_dict[key]], axis=-1)
    
with open('Taobao.pkl', 'wb') as f:
    pickle.dump(user_dict, f)